In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [2]:
# EDA y visualización
movies = pd.read_csv('../final_data/combined_data.csv')

In [3]:
movies.sample(2)

,belongs_to_collection,budget,genres,movie_id,original_language,overview,popularity,production_companies,production_countries,release_date,...,title,vote_average,vote_count,production_countries_code,release_year,release_month,release_day,return,director,actors
3666,X-Men Collection,75000000.0,Action|Adventure|Science Fiction,36657,en,"Two mutants, Rogue and Wolverine, come to a pr...",0.954467,"""Donners Company""|Bad Hat Harry Productions|Ge...",United States of America,2000-07-19,...,X-Men,6.8,4172.0,US,2000,7,19,3.951194,Bryan Singer,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...
24313,The Disappearance Of Eleanor Rigby,0.0,Drama,157829,en,A New York couple's relationship is tested aft...,6.266831,Division Films|Dreambrge Films|Kim and Jim Pro...,United States of America,1981-11-20,...,The Disappearance of Eleanor Rigby: Them,6.2,106.0,US,1981,11,20,0.000000,Ned Benson,Jessica Chastain|James McAvoy|Bill Hader|Viola...


In [4]:
movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'movie_id',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'production_countries_code', 'release_year',
       'release_month', 'release_day', 'return', 'director', 'actors'],
      dtype='object')

#### Seleccion de las columnas

In [5]:
movies = movies[['title','genres','overview','director','release_year']]
movies.sample(2)

,title,genres,overview,director,release_year
24408,Darkside Blues,Animation|Horror|Mystery|Science Fiction,The Persona Century Corporation has purchased ...,Yoshimichi Furukawa,2013
4394,A Night in the Life of Jimmy Reardon,Drama|Romance,A recent high school graduate is faced with tw...,William Richert,1988


In [6]:
movies.genres = movies.genres.str.replace('|',' ')

In [16]:
movies['data_joined'] = movies[['title','genres','director']].astype(str).apply(' '.join, axis=1)
movies.sample()

,title,genres,overview,director,release_year,data_joined
39457,Double Daddy,Thriller,A teen's life turns upside down when her boyfr...,Lee Friedlander,2016,Double Daddy Thriller Lee Friedlander


Cálculo de similitud de puntuación


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
vectorizer = CountVectorizer(stop_words='english')
data_matrix = vectorizer.fit_transform(movies.data_joined)
similarity_matrix = cosine_similarity(data_matrix)

similarity_matrix.shape

MemoryError: Unable to allocate 2.80 GiB for an array with shape (751052712,) and data type int32

In [32]:
# Obtención de películas similares
movie = 'Avatar'

movie_index = movies.loc[movies['title'] == movie].index # Índice de la película de interés
similar_movies = list(enumerate(similarity_matrix[movie_index[0]]))
similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
top_similar_movies = [movies.loc[i, 'title'] for i, _ in similar_movies[1:6]]
similar_movies

[(14540, 0.9999999999999999),
 (1090, 0.7499999999999999),
 (13556, 0.6708203932499369),
 (1193, 0.6681531047810608),
 (1156, 0.6249999999999999),
 (2523, 0.6249999999999999),
 (3748, 0.6249999999999999),
 (9645, 0.6249999999999999),
 (10184, 0.6249999999999999),
 (13245, 0.6123724356957946),
 (528, 0.5892556509887895),
 (582, 0.5892556509887895),
 (1531, 0.5892556509887895),
 (2524, 0.5892556509887895),
 (3867, 0.5892556509887895),
 (4850, 0.5892556509887895),
 (5045, 0.5892556509887895),
 (8856, 0.5892556509887895),
 (9669, 0.5892556509887895),
 (11059, 0.5892556509887895),
 (13134, 0.5892556509887895),
 (15138, 0.5892556509887895),
 (17477, 0.5892556509887895),
 (4752, 0.5773502691896258),
 (4313, 0.5669467095138409),
 (1458, 0.5590169943749475),
 (1516, 0.5590169943749475),
 (1808, 0.5590169943749475),
 (2525, 0.5590169943749475),
 (2605, 0.5590169943749475),
 (4002, 0.5590169943749475),
 (5496, 0.5590169943749475),
 (5749, 0.5590169943749475),
 (7467, 0.5590169943749475),
 (8049, 

In [33]:
top_similar_movies

['The Abyss', 'Dragonball Evolution', 'The Terminator', 'Aliens', 'Superman']

# Segunda forma

In [15]:
movies = movies[['title','genres','overview','director']]

In [16]:
movies['joined_data'] = movies.title.astype(str) + ' ' +  movies.genres.astype(str) + ' '  + movies.director.astype(str)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english') # Elimino las palabras en ingles que se repiten muchas veces

movies.overview = movies.overview.fillna(' ') # Remplazo los valore nulos por espacios vacios pero formato str
tfidf_matrix = tfidf.fit_transform(movies.joined_data)
tfidf_matrix.shape

(45344, 36829)

Como la matriz de similitud es demasiado grande, para prevenir el MememoryError voy a utilizar una tecnica de reduccion de dimensionalidad.

In [27]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarity = linear_kernel(tfidf_matrix[:30000], tfidf_matrix[:30000])

In [119]:
def get_recommendation(movie):
    index_movie = movies.loc[movies.title == movie].index[0]
    sim_scores = list(enumerate(cosine_similarity[index_movie]))  # Obtener una lista de tuplas (índice, similitud)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # Ordeno la lista de manera descendente por el reverse=True en función de los valores de las claves
    sim_scores = sim_scores[1:6] # Me quedo solo con las 5 peliculas mas parecidas. Empieza en 1 porque el 0 es la pelicula del input
    indexes = [m[0] for m in sim_scores]
    similar_movies = [movies.title.iloc[i] for i in indexes]
    return movies.iloc[indexes]

In [127]:
get_recommendation('Batman')

,title,genres,overview,director,joined_data
1326,Batman Returns,Action Fantasy,"Having defeated the Joker, Batman now faces th...",Tim Burton,Batman Returns Action Fantasy Tim Burton
9932,Vincent,Animation Fantasy,Young Vincent Malloy dreams of being just like...,Tim Burton,Vincent Animation Fantasy Tim Burton
18902,Dark Shadows,Comedy Fantasy,Vampire Barnabas Collins is inadvertently free...,Tim Burton,Dark Shadows Comedy Fantasy Tim Burton
2062,Beetlejuice,Comedy Fantasy,"Thanks to an untimely demise via drowning, a y...",Tim Burton,Beetlejuice Comedy Fantasy Tim Burton
6988,Big Fish,Adventure Drama Fantasy,Throughout his life Edward Bloom has always be...,Tim Burton,Big Fish Adventure Drama Fantasy Tim Burton


In [95]:
movie = 'Titanic'

movie_index = movies.loc[movies['title'] == movie].index # Índice de la película de interés
similar_movies = list(enumerate(similarity_matrix[movie_index[0]]))
similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
top_similar_movies = [movies.loc[i, 'title'] for i, _ in similar_movies[1:6]]
similar_movies

[(1636, 1.0),
 (3282, 1.0),
 (25426, 1.0),
 (45228, 1.0),
 (2045, 0.7071067811865475),
 (3281, 0.7071067811865475),
 (16288, 0.7071067811865475),
 (43226, 0.7071067811865475),
 (33368, 0.5),
 (44173, 0.4472135954999579),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.0),
 (56, 0.0),
 (57, 0.0),
 (58, 0.0),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63, 0.0),
 (64, 0.0),
 (65, 0.0

In [96]:
top_similar_movies

['Titanic',
 'Titanic',
 'Titanic 2',
 'The Chambermaid on the Titanic',
 'Raise the Titanic']